In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=1)
search_results = search.invoke("Amrabat Fenerbahçe'ye transfer oldu mu?")
print(search_results)

tools = [search]


[{'url': 'https://www.ntvspor.net/futbol/fenerbahce-nin-resmen-acikladigi-sofyan-amrabat-istanbul-a-geldi-66d1affaec7add04396e0ab4', 'content': 'Fenerbahçe\'den transfere dair yapılan açıklamada, "Kulübümüz, ACF Fiorentina forması giyen Sofyan Amrabat\'ın geçici transferi konusunda kulübü ve oyuncuyla prensip anlaşmasına varmıştır. Sofyan Amrabat sağlık kontrolleri ve ileri transfer görüşmeleri için bugün İstanbul\'a gelecektir" denildi.'}]


In [ ]:
pip install -qU langchain-openai

In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [19]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_fde2829a40', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f531129-2c53-42d6-a839-54be1418fe36-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [10]:
model_with_tools = model.bind_tools(tools)

In [20]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [21]:
response = model_with_tools.invoke([HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Sofyan Amrabat Fenerbahçe transfer'}, 'id': 'call_iR96JGTBCoFwnA3nvh1AHWyG', 'type': 'tool_call'}]


In [14]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools)

In [22]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='9b58d918-aeb0-49d5-8e7e-c7de51a355fa'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-809eee64-8151-4ea4-a61e-49a87ad61d48-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [23]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu? Olduysa ne zaman oldu?")]}
)
response["messages"]

[HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu? Olduysa ne zaman oldu?", id='435a0a87-3642-43db-bf38-8dd69860d55f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hr4AEu6P9wbcAzeMu7hqvsj3', 'function': {'arguments': '{"query":"Sofyan Amrabat Fenerbahçe\'ye transfer oldu mu?"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 102, 'total_tokens': 135}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c03ceb70-f8cc-4533-b399-4f30b72bbd2b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': "Sofyan Amrabat Fenerbahçe'ye transfer oldu mu?"}, 'id': 'call_hr4AEu6P9wbcAzeMu7hqvsj3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 102, 'output_tokens': 33, 'total_tokens': 135}),
 ToolMessage(content='[{"url": "https://www.cnnturk.com/

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-1109ca2a-1833-4b62-a99d-04810fed4eca-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94})]}}
----


In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 106, 'total_tokens': 121}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-f49774ac-7476-4f0a-b635-ae445f4143c8-0', usage_metadata={'input_tokens': 106, 'output_tokens': 15, 'total_tokens': 121})]}}
----
